In [19]:
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm

from autogluon.tabular import TabularDataset, TabularPredictor

import glob

import torch
from torchvision import models, transforms
import cv2
from PIL import Image
from scipy.stats import skew

In [12]:

rootdir = 'C:/Users/user/git/MiraeCity/gesture/labels/txt/'

# Get all txt files in root directory
txt_files = glob.glob(os.path.join(rootdir, '*.txt'))

for txt_file in txt_files:
    # Read txt file
    with open(txt_file, 'r') as f:
        data = f.readlines()
    
    # Split each line into index and label
    records = [line.split() for line in data]
    
    # Convert to DataFrame
    df = pd.DataFrame(records, columns=['index', 'label'])
    
    # Convert txt file name to csv
    csv_file = txt_file.replace('.txt', '.csv')
    
    # Save as csv
    df.to_csv(csv_file, index=False)

In [13]:
print(torch.cuda.is_available())

True


In [2]:
def extract_frames(video_dir, output_dir):
    
    # Get all the .mp4 files in the directory
    videos = glob.glob(os.path.join(video_dir, '*.mp4'))

    # Make sure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over each video
    for video_path in videos:
        
        # Get the video filename without the extension
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        
        # Open the video file
        video = cv2.VideoCapture(video_path)

        # Initialize the frame count
        count = 0

        while True:
            # Read the next frame
            ret, frame = video.read()

            # If the frame is not valid, break the loop
            if not ret:
                break

            # Write the frame to a JPEG file
            frame_name = f"{video_name}_frame{count:04d}.jpg"
            cv2.imwrite(os.path.join(output_dir, frame_name), frame)

            # Increment the frame count
            count += 1

        # Release the video file
        video.release()

In [ ]:
extract_frames('../gesture/data/test', '../gesture/data/test_img')
# extract_frames('../gesture/data/val', '../gesture/data/val_img')

In [3]:
##### data preprocessing
# Instantiate a pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))  # remove the last layer
model.eval()

# Transformation pipeline
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [20]:
# Function to extract features from an image
def extract_features(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Calculate the mean, std dev and skewness of each color channel
    features = []
    for channel in cv2.split(image):
        features.append(np.mean(channel))
        features.append(np.std(channel))
        features.append(skew(channel.ravel()))

    return features

# Directory containing images
image_dir = 'C:/Users/user/git/MiraeCity/gesture/data/test_img'

# Dataframe to hold features
df = pd.DataFrame()

# Loop through each image in the directory
for image_path in os.listdir(image_dir):
    # Get full image path
    full_path = os.path.join(image_dir, image_path)

    # Extract features from the image
    features = extract_features(full_path)

    # Add features to dataframe
    df = df.append(pd.Series(features), ignore_index=True)

# Add column names to the dataframe
df.columns = ['R_mean', 'R_std', 'R_skew', 'G_mean', 'G_std', 'G_skew', 'B_mean', 'B_std', 'B_skew']

# Save the dataframe to a csv file
df.to_csv('image_features.csv', index=False)

C:\Users\user\AppData\Local\Temp\ipykernel_12116\571495589.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(features), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_12116\571495589.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(features), ignore_index=True)


In [21]:
# Directory containing images
image_dir = 'C:/Users/user/git/MiraeCity/gesture/data/test_img'

# Read the existing CSV file
df = pd.read_csv('image_features.csv')

# Add an 'index' column with the file names from the image directory
df['index'] = [os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith('.jpg')]

# Reorder the columns to put 'index' first
df = df[['index', 'R_mean', 'R_std', 'R_skew', 'G_mean', 'G_std', 'G_skew', 'B_mean', 'B_std', 'B_skew']]

# Write the updated DataFrame to a new CSV file
df.to_csv('image_features_with_index.csv', index=False)

In [23]:
# Read the existing CSV file
df = pd.read_csv('image_features_with_index.csv')

# Remove the part after '_frame' from the 'index' column
df['index'] = df['index'].apply(lambda x: x.split('_frame')[0])

# Read the labels from the txt file
labels = pd.read_csv('C:/Users/user/git/MiraeCity/gesture/labels/txt/test_ann.txt', 
                     sep=" ", 
                     header=None, 
                     names=['index', 'label'])

# Remove the '.mp4' extension from the 'index' column in the labels DataFrame
labels['index'] = labels['index'].str.replace('.mp4', '', regex=False)

# Merge the features and labels DataFrames on the 'index' column
df = pd.merge(df, labels, on='index', how='outer')

# Write the updated DataFrame to a new CSV file
df.to_csv('image_features_with_index_and_labels.csv', index=False)

In [ ]:
extract_features('C:/Users/user/git/MiraeCity/gesture/data/test_img')
#extract_features('../gesture/data/val_img')

In [ ]:

# train_df = pd.read_csv('C:/Users/user/git/MiraeCity/gesture/labels/csv/train_ann.csv')
# test_df = pd.read_csv('C:/Users/user/git/MiraeCity/gesture/labels/csv/test_ann.csv')

train_data = TabularDataset(train_df)
test_data = TabularDataset(test_df)

In [18]:
def get_file_type(filename):
    extension = os.path.splitext(filename)[1]
    if extension in ['.jpg', '.jpeg', '.png']:
        return 'image'
    elif extension == '.mp4':
        return 'video'
    else:
        return 'unknown'

# Directory containing images and videos
data_dir = 'C:/Users/user/git/MiraeCity/gesture/data/test_img/'

# Get list of all file names in the directory
file_names = os.listdir(data_dir)

# Create a DataFrame
df = pd.DataFrame(file_names, columns=['filename'])

# Add a column specifying the data type of each file
df['data_type'] = df['filename'].apply(get_file_type)



In [ ]:
#### autogluon
label = 'label'
eval_metric = 'accuracy'
time_limit = 3600 * 0.5 # hrs

predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit, ag_args_fit={'num_gpus': 0, 'num_cpus': 12})